In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string
import numpy as np

In [2]:
dataDF = pd.read_csv("data.csv")

In [3]:
dataDF.head()

,soz_id,sozum,atara,anlami,anahtar,turu2,gosterim_tarihi
0,1,a'dan z'ye (kadar),a dan z ye kadar a A,"baştan aşağı, tamamen, tamamıyla, bütünüyle:<i...","a, A",Deyim,2019-05-20
1,2,aba altında er yatar,aba altında er yatar yatmak,giyim kuşam kişiliğe ölçü olamaz.,yatmak,Atasözü,2019-05-20
2,3,aba gibi,aba gibi abaII,kaba ve kalın (kumaş).,aba(II),Deyim,NaN
3,4,"aba vakti yaba, yaba vakti aba",aba vakti yaba yaba vakti aba vakit,gereksinimler vaktinden önce ve ucuz olduğu za...,vakit,Atasözü,NaN
4,5,abanın kadri yağmurda bilinir,abanın kadri yağmurda bilinir aba kadir bilmek,"bir şeyin gerçek değeri, ancak ona çok ihtiyaç...","aba, kadir, bilmek",Atasözü,NaN


In [4]:
def data_preprocessing(data):
    # Gereksiz sütunları kaldırma
    data = data.drop(columns=["atara","soz_id","gosterim_tarihi"], axis=1)

    # Sütunlar üzerinde işlemler
    for column in data.columns:
        # Kelimeleri küçük harfe dönüştürme
        data[column] = data[column].str.lower()

        # Noktalama işaretlerini silme
        data[column] = data[column].fillna('')
        data[column] = data[column].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

        # Stopwords silme
        sw = stopwords.words('turkish')
        data[column] = data[column].apply(lambda x: " ".join(word for word in str(x).split() if word not in sw))

    return data

In [5]:
dataDF = data_preprocessing(dataDF)

In [6]:
!pip install zeyrek


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [7]:
import zeyrek
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/sewal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
analyzer = zeyrek.MorphAnalyzer()
def get_lemmatize_words(sentence):
    lemmatized_words = []
    for word in sentence.split():
        results = analyzer.analyze(word)
        if results:
            lemma = results[0][0].lemma
            lemmatized_words.append(lemma)
    return lemmatized_words

dataDF["lemmatized"] = dataDF["anlami"].apply(lambda x: " ".join(get_lemmatize_words(x)))

APPENDING RESULT: <(baştan_Adv)(-)(baştan:advRoot_ST)>
APPENDING RESULT: <(baş_Noun)(-)(baş:noun_S + a3sg_S + pnon_S + tan:abl_ST)>
APPENDING RESULT: <(Baştan_Noun_Prop)(-)(baştan:nounProper_S + a3sg_S + pnon_S + nom_ST)>
APPENDING RESULT: <(aşağı_Postp_PCAbl)(-)(aşağı:postpRoot_ST)>
APPENDING RESULT: <(aşağı_Adj)(-)(aşağı:adjectiveRoot_ST)>
APPENDING RESULT: <(aşağı_Adv)(-)(aşağı:advRoot_ST)>
APPENDING RESULT: <(aşağı_Noun)(-)(aşağı:noun_S + a3sg_S + pnon_S + nom_ST)>
APPENDING RESULT: <(aşağı_Noun)(-)(aşağı:noun_S + a3sg_S + pnon_S + dat_ST)>
APPENDING RESULT: <(tamamen_Adv)(-)(tamamen:advRoot_ST)>
APPENDING RESULT: <(tamamıyla_Adv)(-)(tamamıyla:advRoot_ST)>
APPENDING RESULT: <(tamam_Adj)(-)(tamam:adjectiveRoot_ST + adjZeroDeriv_S + noun_S + a3sg_S + ı:p3sg_S + yla:ins_ST)>
APPENDING RESULT: <(ev_Noun)(-)(ev:noun_S + a3sg_S + in:p2sg_S + i:acc_ST)>
APPENDING RESULT: <(ev_Noun)(-)(ev:noun_S + a3sg_S + i:p3sg_S + ni:acc_ST)>
APPENDING RESULT: <(evin_Noun)(-)(evin:noun_S + a3sg_S + pnon

/home/sewal/Desktop/cleaned_data.csv

In [22]:
dataDF2 = pd.read_csv("cleaned_data.csv")

In [24]:
dataDF2.head()

,sozum,anlami,anahtar,turu2,lemmatized
0,adan zye kadar,baştan aşağı tamamen tamamıyla bütünüylei evin...,a a,deyim,baştan aşağı tamamen tamamıyla Unk ev A Z Unk
1,aba altında er yatar,giyim kuşam kişiliğe ölçü olamaz,yatmak,atasözü,giyim kuşam kişilik ölçü olamaz
2,aba,kaba kalın kumaş,abaii,deyim,kaba kalın kumaş
3,aba vakti yaba yaba vakti aba,gereksinimler vaktinden önce ucuz olduğu zaman...,vakit,atasözü,gereksinim vakit önce ucuz olmak zaman karşı
4,abanın kadri yağmurda bilinir,bir şeyin gerçek değeri ancak ona ihtiyaç duyu...,aba kadir bilmek,atasözü,bir şey gerçek değer ancak onmak ihtiyaç duyma...


In [25]:
from transformers import AutoTokenizer, AutoModel
import torch
from tqdm import tqdm


In [26]:
# Model ve tokenizer yükleme
tokenizer = AutoTokenizer.from_pretrained('dbmdz/bert-base-turkish-uncased')
model = AutoModel.from_pretrained('dbmdz/bert-base-turkish-uncased')

In [27]:
# Metinlerinizin embedding'lerini toplu olarak hesaplayan fonksiyon
def get_batch_embeddings(text_list):
    # Tokenizer ve model burada önceden tanımlanmış olmalı
    inputs = tokenizer(text_list, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

In [28]:

# Boş veya geçersiz türde verileri silin
dataDF = dataDF[dataDF['lemmatized'].apply(lambda x: isinstance(x, str) and x.strip() != '')].reset_index(drop=True)

In [30]:
# DataFrame'deki tüm metinleri toplu olarak işleyip embedding'leri hesaplama
batch_size = 32  # Batch size'ı belirleyin, sistem kapasitesine göre artırabilirsiniz
embeddings = []


for i in tqdm(range(0, len(dataDF), batch_size)):
    batch_texts = dataDF['lemmatized'].iloc[i:i+batch_size].tolist()
    batch_embeddings = get_batch_embeddings(batch_texts)
    embeddings.append(batch_embeddings)

100%|█████████████████████████████████████████| 426/426 [05:49<00:00,  1.22it/s]


In [31]:
# Embedding'leri birleştirip DataFrame'e ekleyin
embeddings = torch.cat(embeddings)
dataDF['embedding'] = embeddings.tolist()


In [32]:
# Sonuçları kontrol etmek için DataFrame'i inceleyebilirsiniz
print(dataDF.head())

                           sozum  \
0                 adan zye kadar   
1           aba altında er yatar   
2                            aba   
3  aba vakti yaba yaba vakti aba   
4  abanın kadri yağmurda bilinir   

                                              anlami           anahtar  \
0  baştan aşağı tamamen tamamıyla bütünüylei evin...               a a   
1                   giyim kuşam kişiliğe ölçü olamaz            yatmak   
2                                   kaba kalın kumaş             abaii   
3  gereksinimler vaktinden önce ucuz olduğu zaman...             vakit   
4  bir şeyin gerçek değeri ancak ona ihtiyaç duyu...  aba kadir bilmek   

     turu2                                         lemmatized  \
0    deyim      baştan aşağı tamamen tamamıyla Unk ev A Z Unk   
1  atasözü                    giyim kuşam kişilik ölçü olamaz   
2    deyim                                   kaba kalın kumaş   
3  atasözü       gereksinim vakit önce ucuz olmak zaman karşı   
4  atasözü  b

In [47]:
from sklearn.metrics.pairwise import cosine_similarity


In [48]:
# Embedding'leri listeye dönüştürme
embeddings_list = torch.tensor(dataDF['embedding'].to_list()).numpy()

In [49]:
def get_most_similar(user_embedding, embeddings):
    similarities = cosine_similarity([user_embedding], embeddings)
    most_similar_index = similarities.argmax()
    return most_similar_index, similarities[0][most_similar_index]

In [50]:

def get_lemmatize_words(sentence):
    return [x[0].lemma for x in analyzer.analyze(sentence) if len(x) > 0 and x[0].lemma not in [*string.punctuation, 'Unk']]


In [71]:
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    outputs = model(**inputs)
    # Ortalamayı al ve gradyan hesaplamasından ayır
    return outputs.last_hidden_state.mean(dim=1).detach()

In [72]:
def get_input_embedding(text):
    text = preprocess_input(text)
    text = get_lemmatize_words(text)
    embedding = get_embedding(text)
    
    # Eğer embedding 3D ise, ortalama alarak 2D hale getir
    if embedding.ndim == 3:
        embedding = embedding.mean(dim=1)
    
    return embedding.unsqueeze(0)  

In [75]:
def find_top_n_similar_quotes(input_text, dataDF, n=5):
    input_embedding = get_embeddings(input_text)
    
    # Embedding'leri array'e çevirme
    embeddings = np.array(dataDF['embedding'].tolist())
    
    # Eğer embeddings 3D ise (örneğin [num_samples, seq_length, embedding_dim] gibi), 2D hale getirme
    if embeddings.ndim == 3:
        embeddings = embeddings.mean(axis=1)  # Her bir embedding için ortalamayı alarak 2D hale getiriyoruz
    
    # Kosinüs benzerliği hesaplama
    similarities = cosine_similarity(input_embedding, embeddings)
    
    # Benzerlikleri sıralama ve ilk n öğeyi seçme
    most_similar_indices = np.argsort(similarities[0])[::-1][:n]
    
    # En benzer sözleri ve anlamlarını alma
    top_quotes = dataDF.iloc[most_similar_indices][['sozum', 'anlami']]
    
    return top_quotes

In [83]:
user_input = input("Bir girdi yazın: ")

# En benzer sözü bulma
most_similar= find_top_n_similar_quotes(user_input, dataDF)
print(f"En benzer söz: {most_similar}")

Bir girdi yazın:  kötü insanlara karşı önlem almayı öneren sistem


En benzer söz:                                           sozum  \
2365            bir uyuz keçi bir sürüyü boklar   
10921     selin ağzı tutulur elin ağzı tutulmaz   
607                         al gömlek gizlenmez   
8906   kurtla görüşürsen köpeği yanından ayırma   
8364                               keyif sormak   

                                                  anlami  
2365   kötü yaradılışlı kötü huylu kişi çevresine köt...  
10921  doğal yıkımlara karşı önlem alınır söyledikler...  
607             kötü şeylerin gizlenmesi mümkün değildir  
8906   saldırgan biriyle karşılaşacak olan kişi kendi...  
8364   1 birine iyi misiniz nasılsınız sorularını yön...  
